First notebook for basketball analytics section, can hopefully begin by flagging end of possestion stuff. 


I uploaded the code used by Sarang to calculate the +/- of any player during a game, but it appears not to work exactly the same as before. I'm not sure why, but I better figure that out, how these functions work again, and then perform a similar operation for BPM. 

In [19]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")


In [20]:
# coding: utf-8

# In[1]:

import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)


# In[2]:

def substitution_correction(subs,lineup):
    """Go through a game dataframe. Correct and confirm that the
    substitution team assignments are correct.

    Parameters
    ----------

    subs : df
        pandas dataframe of every substitution to occur during a game.

    lineups : df
        provided data, the lineup for that given game

    Returns
    -------

    subs : df
        Corrected version of subs where the
        team assignments for the player are correct.

    """
    #Step 1 of cleaning, delete extra cols and make a person1 col to merge w/ subs.
    lineup['Person1'] = lineup['Person_id']
    del lineup['Period'],lineup['status'],lineup['Game_id']
    del subs['Team_id']
    first = pd.merge(subs, lineup,
        on = ['Person1'], how = 'left')
    
    first.drop_duplicates(inplace=True)
    del lineup['Person1']

    weird = first[['Person2','Team_id']]  #now act in parralel. make a df of "first" with only nan cols.
                                    #and weird has every col except but just p2 and team.
    nanmask = first['Team_id'].isna().values
    weird = weird[~nanmask]

    nanfirst = first[nanmask]
    nanfirst.dropna(axis=1,inplace=True) #remove columns to be added back soon
    weird['Person1'] = weird['Person2']
    weird = weird[['Person1','Team_id']]


    second = pd.merge(nanfirst, weird,
        on = ['Person1'], how = 'left')
    del first['Person_id']
    second.drop_duplicates(inplace=True) #remove extraneious cols and shove this into the first[nanmask]!
    if len(second.values) >0:
        first[nanmask] = second.values #and that's it.


    subs = first[['Game_id','Event_Num','Event_Msg_Type','Period','WC_Time','PC_Time','Action_Type','Option1','Option2',
                'Option3','Team_id','Person1','Person2','Team_id_type','Event_Msg_Type_Description']]

    corrected_subs = subs
    return corrected_subs



def sub(playersin, bench, substitution):
    """Function to calculate plus/minus for individuals when they subout
    and calculate difference when subbing in.

    Parameters
    ----------

    playersin : df
        dataframe of players on the court, has columns 'Team_id', 'Person_id', diffin
        and 'pm'

    bench : df
        dataframe of players on the bench, has columns 'Team_id', 'Person_id', diffin
        and 'pm'

    substitution : df
        a single row from the play-by-play dataframe for which the substitution occurs
        Event Msg Type: 8

    Returns :
    playersin, bench
    -------
    """

    teamid = substitution['Team_id']
    suboutID = substitution['Person1']
    subinID = substitution['Person2']
    score1 = substitution['score_x']
    score2 = substitution['score_y']

    if teamid == teams[0]:
        diff = score1 - score2
    else:
        diff = score2 - score1

    suboutindex = (playersin['Person_id'] == suboutID)
    playersin.loc[suboutindex,'pm'] = playersin.loc[suboutindex,'pm']     + diff - playersin.loc[suboutindex,'diffin']

    if ~bench['Person_id'].str.contains(subinID).any(): # if the player isn't in the "bench" df,
                                                        # ie. hasn't already appeared in the game
            playersin = playersin.append(
            {'Team_id': teamid, 'Person_id':subinID, 'diffin':diff, 'pm':0},
            ignore_index = True)
    else: # if he's in the "bench" df already
        playersin = playersin.append(bench.loc[bench['Person_id'] == subinID])
        bench = bench.loc[~(bench['Person_id'] == subinID)]

    bench = bench.append(playersin.loc[playersin['Person_id'] == suboutID])
    playersin = playersin.loc[~(playersin['Person_id'] == suboutID)]

    # set the score difference for new player
    playersin.loc[playersin['Person_id'] == subinID, 'diffin'] = diff

    return playersin, bench


# In[4]:

def startperiod(playersin, bench, startrow):
    """ Function to switch out players at the start of the period
    and calculate the score difference coming in

    Parameters
    ----------

    playersin : df
        dataframe of players on the court, has columns 'Team_id', 'Person_id', diffin
        and 'pm'

    bench : df
        dataframe of players on the bench, has columns 'Team_id', 'Person_id', diffin
        and 'pm'

    startrow : df
        a single row from the play-by-play dataframe for which the start of the period occurs
        Event Msg Type: 12

    Returns 
    -------

    playersin, bench
    """

    score1 = startrow['score_x']
    score2 = startrow['score_y']
    diff = score1 - score2
    period = startrow['Period']

    # identify who is coming in at the start of the period
    periodstarters = lineup.loc[(lineup['Game_id'] == game) & (lineup['Period'] == period)]
    allplayers = pd.concat([bench, playersin])

    # allocate players going in and those on the bench
    playersintemp = pd.concat([
            playersin.loc[playersin['Person_id'].isin(periodstarters['Person_id'])], \
            bench.loc[bench['Person_id'].isin(periodstarters['Person_id'])]
        ])
    benchtemp = pd.concat([
            bench.loc[~bench['Person_id'].isin(periodstarters['Person_id'])], \
            playersin.loc[~playersin['Person_id'].isin(periodstarters['Person_id'])]
        ])

    playersin = playersintemp
    bench = benchtemp

    # check to see if there are players first coming in at the start of the period
    check = periodstarters['Person_id'].isin(allplayers['Person_id'])
    if ~check.all():
        newplayers = periodstarters.loc[~check]
        for index,newplayer in newplayers.iterrows():
            playersin = playersin.append(
                {'Team_id': newplayer['Team_id'],
                 'Person_id': newplayer['Person_id'],
                 'diffin':0,
                 'pm':0},
                ignore_index = True)

    # set the score difference for all players at the start of the period
    playersin.loc[playersin['Team_id'] == teams[0],'diffin'] = diff
    playersin.loc[playersin['Team_id'] == teams[1],'diffin'] = -diff


    return playersin, bench


def endperiod(playersin, bench, endrow):
    """ Function to calculate the plus minus for everyone at the end of the period

    Parameters
    ----------

    playersin : df
        dataframe of players on the court, has columns 'Team_id', 'Person_id', diffin
        and 'pm'

    bench : df
        dataframe of players on the bench, has columns 'Team_id', 'Person_id', diffin
        and 'pm'

    startrow : df
        a single row from the play-by-play dataframe for which the start of the period occurs
        Event Msg Type: 12

    Returns :
    playersin, bench
    -------
    """

    score1 = endrow['score_x']
    score2 = endrow['score_y']
    diff = score1 - score2

    # calculate plus minus for everyone at the end of the period
    playersin.loc[playersin['Team_id'] == teams[0], 'pm']         = playersin.loc[playersin['Team_id'] == teams[0],'pm']         + diff - playersin.loc[playersin['Team_id'] == teams[0],'diffin']
    playersin.loc[playersin['Team_id'] == teams[1], 'pm']         = playersin.loc[playersin['Team_id'] == teams[1],'pm']         - diff - playersin.loc[playersin['Team_id'] == teams[1],'diffin']

    return playersin, bench

In [21]:
#load in data. 
pbp = pd.read_csv('Basketball Analytics/Play_by_Play.txt',delimiter='\t')
lineup = pd.read_csv('Basketball Analytics/Game_Lineup.txt',delimiter='\t')
ec = pd.read_csv('Basketball Analytics/Event_Codes.txt',delimiter = '\t')


pbp = pd.read_csv('Basketball Analytics/Play_by_Play.txt',delimiter='\t')
lineup = pd.read_csv('Basketball Analytics/Game_Lineup.txt',delimiter='\t')
codes = pd.read_csv('Basketball Analytics/Event_Codes.txt',delimiter = '\t')




In [32]:
pms = pd.DataFrame([])

#pt 2. isolate one game instance
# indentify sample games, loop through all
sample_games = pbp['Game_id'].unique()
######Loop and test sub function here ######
for game_num in range(len(sample_games))[0:1]:
    game = sample_games[game_num] # for now, not iterative.
    print("game id: ", game)
    print("game num: ",game_num)
    teams = lineup.loc[lineup['Game_id'] == game]['Team_id'].unique()
    
    pbp_singlegame = pbp.loc[pbp['Game_id'] == sample_games[game_num]].sort_values(['Period','PC_Time','Event_Msg_Type','WC_Time','Event_Num'],
        ascending=[True,False,True,True,True])
    pbp_singlegame = pbp_singlegame.merge( codes,
        on = ['Event_Msg_Type', 'Action_Type'], how = 'left')
    pbp_relevant = pbp_singlegame #.loc[(pbp_singlegame['Event_Msg_Type'] == 1) |
        #(pbp_singlegame['Event_Msg_Type'] == 3) | (pbp_singlegame['Event_Msg_Type'] == 8) |
       # (pbp_singlegame['Event_Msg_Type'] == 12) | (pbp_singlegame['Event_Msg_Type'] == 13)]
   # del pbp_relevant['Action_Type_Description']
  
    starting_lineup = lineup.loc[(lineup['Game_id'] == game)] #starting lineup of the game
    subsmask = pbp_relevant['Event_Msg_Type'] == 8
    subs = pbp_relevant[subsmask] #dataframe of all substitutions. 
    playersin = pd.DataFrame(starting_lineup.loc[(starting_lineup['Period'] == 1),['Team_id','Person_id']])

    subs_correct = substitution_correction(subs,starting_lineup)
    pbp_relevant[subsmask] =  subs_correct

    
    playersin['ortg'] = 0 #initialize offensive rating
    playersin['diffin'] = playersin['pm'] = 0
    bench = pd.DataFrame(columns = ['Team_id', 'Person_id', 'diffin', 'pm','ortg']) #again added that column.


    pbp_relevant.loc[(pbp_relevant['Event_Msg_Type'] == 3)&(pbp_relevant['Option1'] != 1), 'Option1'] = 0
    pbp_relevant['score'] = pbp_relevant.groupby('Team_id', axis = 0)['Option1'].cumsum() #calulate score of each team throughout the game. 
    team1score = pbp_relevant.loc[pbp_relevant.iloc[:,10] == teams[0]] #score of that team at a given time. 
    team2score = pbp_relevant.loc[pbp_relevant.iloc[:,10] == teams[1]]
    del pbp_relevant['score']

    pbp_relevant = pd.merge(pbp_relevant, team1score.loc[:,['Event_Num','score']], on = 'Event_Num', how = 'left')
    pbp_relevant = pd.merge(pbp_relevant, team2score.loc[:,['Event_Num','score']], on = 'Event_Num', how = 'left')

    # fill forward for NaNs
    pbp_relevant.loc[:,['score_x', 'score_y']] = pbp_relevant.loc[:,['score_x', 'score_y']].fillna(method = 'ffill')
    pbp_relevant.loc[:,['score_x', 'score_y']] = pbp_relevant.loc[:,['score_x', 'score_y']].fillna(0) # for the start of the game
    del team1score, team2score
    
    # Loop through play by play
    for index, row in pbp_relevant.iterrows():
        if (row['Event_Msg_Type'] == 8):
            playersin, bench = sub(playersin, bench, row)  #calculate +/- of subout.
        elif (row['Event_Msg_Type'] == 13):
            playersin, bench = endperiod(playersin, bench, row)  #calculate +/- at end of period,
          #  break

        elif (row['Event_Msg_Type'] == 12):
            playersin, bench = startperiod(playersin, bench, row) #update lineups
        
    pm = pd.concat([bench,playersin])
    pm = pm.sort_values(['Team_id','pm'])
    pm = pm.reset_index(drop=True)

    pm['game_id'] = game
    pms = pms.append(pm)
    print(pm['pm'].sum())


# In[ ]:

pms = pms[['game_id','Person_id','pm']]
pms = pms.rename(index = str, columns = {'game_id': "Game_ID",'Person_id': 'Player_ID','pm': 'Player_Plus/Minus'})


# In[14]:

pms

game id:  006728e4c10e957011e1f24878e6054a
game num:  0
0.0


,Game_ID,Player_ID,Player_Plus/Minus
0,006728e4c10e957011e1f24878e6054a,42e0d7167f04a4ff958c6442da0e6851,-18
1,006728e4c10e957011e1f24878e6054a,f4a5ca938177c407a9dab5412e39498f,-18
2,006728e4c10e957011e1f24878e6054a,c5dd5b2e3b975f0849d9b74e74125cb9,-14
3,006728e4c10e957011e1f24878e6054a,2ad626904c8b28cceb8e12c624a84240,-14
4,006728e4c10e957011e1f24878e6054a,5db9c1c8184510fee8161e7fafdc9c49,-14
5,006728e4c10e957011e1f24878e6054a,766802a8fda500d7945950de7398c9c6,0
6,006728e4c10e957011e1f24878e6054a,618f6d58ab2881152607c2a6e057bc51,0
7,006728e4c10e957011e1f24878e6054a,8d2127290c94bd41b82a2938734bc750,3
8,006728e4c10e957011e1f24878e6054a,ae53f8ba6761b64a174051da817785bc,5
9,006728e4c10e957011e1f24878e6054a,99104de2626f67c1fa2ce70504970c3f,-5


In [33]:
def end_of_possession_flag(z):
    """Creates a 1 or 0 flag to denote whether or not this event code corresponds to the end of a possession.
    
    
    An idea based on this is to then make a running count of this similar to points, get the total # of possessions based on points scored. 
    
    """
    
    
    if z['Event_Msg_Type_Description'] == 'Made Shot':
        return 1 #this is one way to end the possesison, the shot goes in and possesion changes. 
    
    if z[]

SyntaxError: invalid syntax (<ipython-input-33-f50bc5ee8864>, line 13)

In [34]:
pbp_relevant

,Game_id,Event_Num,Event_Msg_Type,Period,WC_Time,PC_Time,Action_Type,Option1,Option2,Option3,...,Person2,Person3,Team_id_type,Person1_type,Person2_type,Person3_type,Event_Msg_Type_Description,Action_Type_Description,score_x,score_y
0,006728e4c10e957011e1f24878e6054a,4.0,10.0,1.0,546420.0,7200.0,0.0,0.0,0.0,0.0,...,99104de2626f67c1fa2ce70504970c3f,766802a8fda500d7945950de7398c9c6,2.0,4.0,5.0,4.0,Jump Ball,,0.0,0.0
1,006728e4c10e957011e1f24878e6054a,2.0,12.0,1.0,546410.0,7200.0,0.0,0.0,0.0,0.0,...,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,0.0,0.0,0.0,0.0,Start Period,,0.0,0.0
2,006728e4c10e957011e1f24878e6054a,7.0,1.0,1.0,546590.0,7020.0,79.0,2.0,0.0,0.0,...,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2.0,4.0,0.0,0.0,Made Shot,Pullup Jump shot,2.0,0.0
3,006728e4c10e957011e1f24878e6054a,10.0,3.0,1.0,546810.0,7020.0,10.0,1.0,0.0,0.0,...,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2.0,4.0,0.0,0.0,Free Throw,Free Throw 1 of 1,3.0,0.0
4,006728e4c10e957011e1f24878e6054a,8.0,6.0,1.0,546620.0,7020.0,2.0,0.0,0.0,1.0,...,42e0d7167f04a4ff958c6442da0e6851,0b1c631effab25d4af2d16fc123c3d31,2.0,5.0,4.0,1.0,Foul,Shooting,3.0,0.0
5,006728e4c10e957011e1f24878e6054a,11.0,1.0,1.0,546970.0,6900.0,79.0,2.0,0.0,0.0,...,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,3.0,5.0,0.0,0.0,Made Shot,Pullup Jump shot,3.0,2.0
6,006728e4c10e957011e1f24878e6054a,12.0,2.0,1.0,547190.0,6680.0,63.0,2.0,0.0,0.0,...,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2.0,4.0,0.0,0.0,Missed Shot,Fadeaway Jump Shot,5.0,2.0
7,006728e4c10e957011e1f24878e6054a,13.0,4.0,1.0,547200.0,6660.0,0.0,0.0,0.0,0.0,...,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2.0,5.0,0.0,0.0,Rebound,Unknown,5.0,2.0
8,006728e4c10e957011e1f24878e6054a,14.0,2.0,1.0,547260.0,6610.0,41.0,2.0,1.0,0.0,...,0370a0d090da0d0edc6319f120187e0e,8d2127290c94bd41b82a2938734bc750,3.0,5.0,0.0,4.0,Missed Shot,Running Layup Shot,5.0,4.0
9,006728e4c10e957011e1f24878e6054a,15.0,4.0,1.0,547280.0,6610.0,0.0,0.0,0.0,0.0,...,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,3.0,4.0,0.0,0.0,Rebound,Unknown,5.0,4.0


In [26]:
ec

,Event_Msg_Type,Action_Type,Event_Msg_Type_Description,Action_Type_Description
0,1,0,Made Shot,No Shot
1,1,1,Made Shot,Jump Shot
2,1,2,Made Shot,Running Jump Shot
3,1,3,Made Shot,Hook Shot
4,1,4,Made Shot,Tip Shot
5,1,5,Made Shot,Layup Shot
6,1,6,Made Shot,Driving Layup Shot
7,1,7,Made Shot,Dunk Shot
8,1,8,Made Shot,Slam Dunk Shot
9,1,9,Made Shot,Driving Dunk Shot


What is a possession?  Any time there is a made shot. 

Made free throw x of x. 

Defensive rebound

turnover

end of period. 







In [38]:
# sort out play by play for singe game
    pbp1 = pbp.loc[pbp['Game_id'] == sample_games[game_num]]
#Properly sorted format.
   # pbp1 = pbp1.sort_values(['Period','PC_Time','WC_Time','Event_Num'],
   #     ascending=[True,False,True,True])

#New theory for sorting...
    pbp1 = pbp1.sort_values(['Period','PC_Time','Event_Msg_Type','WC_Time','Event_Num'],
        ascending=[True,False,True,True,True])
#Here would be a good place to add FTE
#     pbp1 = pbp1.reset_index(drop=True)

#translate to understand what each play is.
    pbp1 = pd.merge(pbp1, codes,
        on = ['Event_Msg_Type', 'Action_Type'], how = 'left')

# subset out extraneous plays, only need scoring plays, free throws, subsitions
# start and end of period
    pbpx = pbp1.loc[(pbp1['Event_Msg_Type'] == 1) |
        (pbp1['Event_Msg_Type'] == 3) | (pbp1['Event_Msg_Type'] == 8) |
        (pbp1['Event_Msg_Type'] == 12) | (pbp1['Event_Msg_Type'] == 13)]
    del pbpx['Action_Type_Description']
    lineup1 = lineup.loc[(lineup['Game_id'] == game)]
    subsmask = pbpx['Event_Msg_Type'] == 8
    subs = pbpx[subsmask]
    playersin = pd.DataFrame(lineup1.loc[(lineup1['Period'] == 1),['Team_id','Person_id']])
    subs_correct = substitution_correction(subs,lineup1)
  #  print( subs_correct.isna().sum())
    pbpx[subsmask] =  subs_correct.values


    playersin['diffin'] = playersin['pm'] = 0
    bench = pd.DataFrame(columns = ['Team_id', 'Person_id', 'diffin', 'pm'])
    #### Cleaning now complete, now calculating pm for every game.

    pbpx.loc[(pbpx['Event_Msg_Type'] == 3)&(pbpx['Option1'] != 1), 'Option1'] = 0
    pbpx['score'] = pbpx.groupby('Team_id', axis = 0)['Option1'].cumsum()
    team1score = pbpx.loc[pbpx.iloc[:,10] == teams[0]]
    team2score = pbpx.loc[pbpx.iloc[:,10] == teams[1]]
    del pbpx['score']

    pbpx = pd.merge(pbpx, team1score.loc[:,['Event_Num','score']], on = 'Event_Num', how = 'left')
    pbpx = pd.merge(pbpx, team2score.loc[:,['Event_Num','score']], on = 'Event_Num', how = 'left')

    # fill forward for NaNs
    pbpx.loc[:,['score_x', 'score_y']] = pbpx.loc[:,['score_x', 'score_y']].fillna(method = 'ffill')
    pbpx.loc[:,['score_x', 'score_y']] = pbpx.loc[:,['score_x', 'score_y']].fillna(0) # for the start of the game
    del team1score, team2score

    # Loop through play by play
    for index, row in pbpx.iterrows():
        if (row['Event_Msg_Type'] == 8):
            playersin, bench = sub(playersin, bench, row)  #calculate +/- of subout.
        elif (row['Event_Msg_Type'] == 13):
            playersin, bench = endperiod(playersin, bench, row)  #calculate +/- at end of period,
        elif (row['Event_Msg_Type'] == 12):
            playersin, bench = startperiod(playersin, bench, row) #update lineups

    pm = pd.concat([bench,playersin])
    pm = pm.sort_values(['Team_id','pm'])
    pm = pm.reset_index(drop=True)

    pm['game_id'] = game
    pms = pms.append(pm)
    print(pm['pm'].sum())


# In[ ]:

pms = pms[['game_id','Person_id','pm']]
pms = pms.rename(index = str, columns = {'game_id': "Game_ID",'Person_id': 'Player_ID','pm': 'Player_Plus/Minus'})


# In[14]:

pms


# In[15]:

game id:  006728e4c10e957011e1f24878e6054a
game num:  0


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

ValueError: Must have equal len keys and value when setting with an ndarray

In [39]:
subsmask

1      False
2      False
3      False
5      False
10     False
11     False
14     False
20     False
23     False
25     False
35     False
37     False
38     False
39     False
40     False
41     False
59     False
62      True
63      True
65     False
70     False
71      True
72      True
73      True
86      True
87      True
88     False
89     False
96      True
97     False
98     False
99     False
101     True
106    False
107    False
108    False
116    False
117    False
118    False
119    False
121    False
125    False
138    False
139     True
141    False
142    False
145    False
147     True
148     True
149     True
151    False
152    False
159    False
165    False
167     True
168     True
169    False
171    False
172    False
176     True
177     True
178    False
181    False
182     True
188    False
189    False
191    False
192    False
194    False
195    False
196    False
197    False
202    False
204     True
205    False
216    False
217    False